# INSTALL: packages

In [3]:
import WalletClustering_MIH
import pandas as pd
import pickle
from io import StringIO

# EXECUTE: MIH on list of additional addresses to cluster

In [ ]:
otherServicesList = StringIO('''address,otherService
152f1muMCNa7goXYhYAQC61hxEgGacmncB,btccpool
1CK6KHY6MHgYvmRQ4PAafKYDrg1ejbH1cE,slushpool.com
14xpJvJD8G3PBPWaT8AdBWuZUpsdV5j8RH,slushpool.com
1AqTMY7kmHZxBuLUR5wJjPFUvqGs23sesr,slushpool.com
1CjPR7Z5ZSyWk6WtXvSFgkptmpoi4UM9BC,ghash.io
15urYnyeJe3gwbGJ74wcX89Tz7ZtsFDVew,antpool.com
1GuMujABuc8kvzDTyVJFpcf4vszPUgsjiU,antpool.com
1Mp3atombUR6iLLYoLEdN9uPfPscsKBXfi,antpool.com
19PkHafEN18mquJ9ChwZt5YEFoCdPP5vYB,bitminter.com
18M9o2mXNjNR96yKe7eyY6pfP6Nx4Nso3d,eclipsemc.com
1NfJfjsyfxJ9zoA1LPj4TdDHf8mSUvQ17r,eclipsemc.com
187b1MZr73iuPGnABu6d4eSYmt2NQHkTqt,eclipsemc.com
1B55K4WuR1AQKCmF8WznDmhZNw6G1x8uEc,eclipsemc.com
1H1sq6Msgt9HjRrBuz8ieZdThzWXS6oPVA,kncminer.com
1FeDtFhARLxjKUPPkQqEBL78tisenc9znS,bitfury.org
1JLRXD8rjRgQtTS9MvfQALfHgGWau9L9ky,bw.com
1JkT6EQMotCz6BFqPWFd3kFDQzU1qa119N,eligius.st
1N6LrEDiHuFwSyJYj2GedZM2FGk7kkLjn,kano.is
13pucx6gHP2vyBLc88QfcGivjkhK63PeVg,kano.is
1DXRoTT67mCbhdHHL1it4J1xsSZHHnFxYR,telco214
37C4WW3t4YkqehPHTxqUbaEa8o97W66XdW,coinpayments.net
3NSAY3HF6Afd4KyEoYUiQdkJFLfr4E7ubj,xapo.com
1CAxhfNvk1xEtMuS4J8b1TrZkifReevW5s,cubits.com
3CF9DbG6nvCUTjK2xBdX9uYkaDyM9mZyWH,cryptonator.com
16ttqBTMimk3w6veGF3MV4t2Ln56SiFhPc,cryptonator.com
1ArJTD4iR3SCwjuyiP3H6pbwvPBHduX6WB,bitpay.com
1NgiUwkhYVYMy3eoMC9dHcvdHejGxcuaWm,bitpay.com
115gF9GA8qfxEXp9UjhVzRUwp3yvvvu6hN,bitpay.com
1QDMykW6y1v5aUUBAoXjRUBMzC7euEEVts,bitpay.com
18Xmh94gLPSBYFRJBtezD9awaRRnj2QvCL,bitoex.com
13hQVEstgo4iPQZv9C7VELnLWF7UWtF4Q3,haobtc.com
3AwLzYLQvzNdBA3qHPCDbTcrJGBXLs1k9c,cryptopay.me
1HrHsgAuHrHYJHVWPpgm8YkjtcyFxBSBKt,cryptopay.me
14geEFfPXT9K5Vao9DcoFeikPDJyc7QZUM,alphabaymarket
17DEJSAMHsTtsxZA1v6yu51xVwtFJ5ECfp,alphabaymarket
1LDM7MdmimPdyKZLZnmJoRUqRcjHMsakkC,nucleusmarket
3NcasnmxegbuMw2CK1AA6oSNDG33ZYivwQ,bitcoinfog
32vS9wnvG9gGRwJQhk9hkGSZvhy1u7Myt5,bitcoinwallet.com
1E5iM4SRuQraWaetjUpyRVkTQRZ8pUZxhg,coinjar.com
3ETx7r3qNkFADxBt7DJWqwk9JDVJiu8Tx6,holytransaction.com
1NP5HCMVBWArhi4Y5yL81VySEvLYVp1zVW,btcjam.com
1KzswLrDKziFHMzQFp63fS12aMzyJdTBtj,vip72.com
3Gbb1aeerHrqgTRQfa6xBfSD3J9HhbFKsU,moonbit.co.in
1K17CWKGb74yk6C64wQ9iyrdgUwN6ifNQw,coinkite.com
1AUvrDGLJNeGFQ5RL5HAxzCyvq2fXsinTS,faucetbox.com
19FbJLrD6BjaS1bh5jHHvNyyGFQRTj6waL,oklink.com
1PBxY3VH7w4PmbRG4nJBJYwZt92Mc9gn6U,purse.io
1PY3ziMwTMwDLryXdb3WsfPaN637jpwXbv,epay.info
12fnqJqEfEjTVnkhEPYjhXrRD1i6bZZeZg,loanbase.com
1ALRsFWtAoetAaht3yQ7pwhwmNSiunxMxR,crimenetwork.co
17EBXubB23UsDJ7oknryQHHWT7NejttQZy,crimenetwork.co
1EPHNmtCnWZU4A7Vm68TDHmKrHoGaQ9GuA,germanplazamarket
1E4BZfG7uLMivuRLTa1k2CVQDUPmvpxGXJ,bitbond.com
1MS17CwusivpXaBSf1CgQ9VcnRdco4FwUm,paymium.com
19vfz2AJj4kSkvC9t2mLg5sgtkLTk7Kwwu,strongcoin.com-fee
1H2wHJ1hPtX7GQv8NZ7HdAYYQg8UzNhfJo,cryptostocks.com
3Pu3SEE5hEdQTvLWWXBiBBmWzQNpyoTq5f,coinapult.com
12QFkzXR7vGRb2JkLo1YvsP1ioiADZJWQp,coinapult.com
1MpAJJ3z1SuKzUpstRWyWJGvwnpyTzD3Nf,genesis-mining.com
1AwLahmayYFYY2qWBciXh2KKFjRKqc5Urr,changetip.com
1MkDc8o82PEVhRcW2uxEDdpjHrRrTVs9rU,doctordmarket
12sUDSyw79pxFP1nP2aB519VXF9noAsgaG,gocelery.com
1KLktTKdGXkQGtL788ZdJt8bBT9Kr9ySjJ,btcpop.co
1Efd2nct3eUijTLmmxRxcPWUnUJ8YDVLbr,btclend.org
1GauTCnH1EgHvGoqAs9EhMY7P7yq2ANoSA,coinurl.com
1BrF8iYUJAzngDV2j2sU7dmnMhBmTB7NJs,bitnz.com
1cointQVgw2EwnJx3EFVPvD65gSsD9nJ7,coinbox.me
1Codxq4pdK72K6vDMQEhhmat2eVCcJ1XUX,coinworker.com
146QyVLdpsLmdbzswtwkuWboNt8YqRJZji,watchmybit.com
1ET9RCoy4yfRxSXEnpHibKzp8mfqDMikMb,bitlaunder.com
1ABEzWJpiKzzhy8wyUHUKevkB1pdcL9vXv,bitclix.com
147VmekydXM2JvJpX5sXbYWgGkEHVUgEHa,vic-socks.to
1Cw3xr6ZxBKEzCSTkCwryip5GU6MNLHpPZ,securevpn.to
1Bvuydv84ZMdzrHfrdThdgLKqEdG14pGsx,securevpn.to
14jpsTvys7mqecHC851xJpj6wF6pQDyaXF,bylls.com
1Q99AKghHJ3VEWA6UGTWJrE7icSWRhozRF,greenroadmarket
13FekTJqJevuSvry3rnYHqTC1tVEEZPHYU,satoshidice.com
1dice8EMZmqKvrGE4Qc9bUFf9PX3xaYDp,satoshidice.com
1NxaBCFQwejSZbQfWcYNwgqML5wWoE3rK4,luckyb.it
161gTqw3CHVqiBfmC3vCLrXPob6wLQSKP4,luckyb.it
1KK3jqBM8koV5caHuYJ3ftxsiFPrUXvyYf,bitzillions.com
143WdLNY4oyQURUUx66G5Z9ZZaiV3fGqhS,999dice.com
1D3drM1tF4WxGphjPGqQcHSscrsbz8kgxC,cloudbet.com
1LkEiNuK47XdoQLiMBhYEEZvz3iUDqQhQP,coingaming.io
1H2s3ZSXuYKKXp8SvPki851FghJq5q2sH4,primedice.com
1Bm8BfJPGZaaDTAwqP2pE4twsR1oyTjw1y,primedice.com
15CWJpAe9rLFLQPrsqPiViu2iS4yUEKFEh,primedice.com
1MsgD1v4dqGPkCjbBoLHZqJNybyMXZNDf5,primedice.com
11223UDswFBFbMJ75r3K8epaqX6ZxQpPVd,primedice.com
1545LBUF2CZCkwbrUuaedgNskcXtdbhsYV,satoshimines.com
1EDLW1yYXciLunPs2o1hCsiM3v6HwStvbR,nitrogensports.eu
1ARF7D3a1CTUP2ej3UNuLSY54VnftKNfPm,secondstrade.com
1Nrj7LXt6pzVGd9q6vWpFy2pkuDrsfxNwZ,pocketdice.io
1G3tJZeNRRKXqjisp21i76FHmDE9ToY6HZ,fortunejack.com
1BTS4wdHD2FZ7Srbh8aaFuwVF1zVi7oDfD,rollin.io
12DzK5gjSEYbmBZUn3XkQ9ypmytHNpvjDC,bitzino.com
1KpnaXqHgEut1LCaE1maLZBKgAgEu8G7QF,bitcoinvideocasino.com
1MkuYk9nk19Wyh3k285mfo9ru8S9vVt7G4,bitcoinvideocasino.com
1MWwPwmJyPfCMtq9fDcxiYhXMxndeZtjfL,bitcoinvideocasino.com
1AapX7nstee5o5BUfNJVHCm28m7oYuKvhi,yabtcl.com
17hiCXtpyFKXMSzGD3Tt1xqfTFK5Jt9mWB,betcoin.ag
1JzVYbjEA49w6zMhKJ6PssUoKpooK6KKgF,betcoin.ag
1AX18zUCf1CwW4PNPXa6b7kmJPgM6vFAs9,satoshibet.com
1p8J4v54f3XhEJWzRoHG8Ayxw5cfrcyj1,safedice.com
1BKSARZdi7vyBvTv4LRW9QGe3cvfuCBjsz,coinroll.com
1JKhUULaWYnaNUouSkKbXEmqCBMB61pnQq,crypto-games.net
1BTPSjiSC1s3A5cqJrbDRi2LAHx3U88mKR,betcoin.tm
1AhpCo63MF5ayyK7H3E98zoPhMYMVj8sH1,swcpoker.eu
13Lyp8iawQyyychG6aHe71UkkNp53cR6DW,satoshiroulette.com
1Up15Msx4sbvUCGm8Xgo2Zp5FQim3wE59,btcoracle.com
1DwmVmbMetcvvR1j2Tj4o3QCG9VMwFjjcY,peerbet.org
1AMqnsRgbMjqZLPqBQPuuX7THgorF8WngM,anonibet.com
1Jsi3rotP9FXE7tKbwo83WYAeUuTcM98Hr,satoshi-karoshi.com
14mXxHFQeUYjQMraNMsB6hPFhwsAQSMZqj,satoshi-karoshi.com
1E421ChpHWEqQ2Kcy2JqEE8eDx7nFN4AXj,777coin.com
1Q5X43DKVF1R8XM1MsyquCaNmNrzzmaReV,bitstarz.com
1GMJ1LExqYtC7HWkjT2KRLZuhfNzpFFKUM,satoshicircle.com
1CeqPhoTFbX8zq3wWCFUbt7g5C3TDbQFsD,coinichiwa.com
1FFwxD2C7DUohz4m5BW7h4yzCqm48dBPHd,coinroyale.com
1M41nU3P7apHs7KtyjnZaegEBSyL5EaX9m,coinroyale.com
16PDHmBRKLxCvzttLJvYjj2aDVPryt54a2,coinroyale.com
1EKfcwM8Kur9A7fJnU5ZE5cE7DcdiKR3ig,betmoose.com
1KTrWTMsB7zYADPFSCPnisBEYYzAhLXsdN,jetwin.com
1KLDzdqBveuSLjf7hJfBzpKhoZ7JrENASS,betchain.com
1Gko7FZ3S5btD837ryZrkJvmQ9SfYwT5kW,bitcoinpokertables.com
15rM6EwpqdSG3CX1ZGK5VeLjQAXZ5zg1z3,dicenow.com
18aEhkGnYYc3wxMeQELiN2rSKpHgLKG1MY,fairproof.com
1CYju8vZYKytTQDmvRxBggeVYRPSvAFmoi,diceoin.io
19mS9jJD5srCNwDVJetQRrUYnnhWQGmJc,minefield.bitcoinlab.org
''')

In [ ]:
# 37C4WW3t4YkqehPHTxqUbaEa8o97W66XdW,coinpayments.net is big! ca. 10mio addresses

In [ ]:
otherServicesListDf = pd.read_csv(otherServicesList,sep=",")

In [ ]:
# iterate over list of addresses with MIH
for index, row in otherServicesListDf.iterrows():
    start_address = otherServicesListDf['address'][index]
    association = otherServicesListDf['otherService'][index]
    iterMultiInputClustering_chunks(start_address, association)

In [ ]:
  # export otherServicesListDf to file for assocBlacklist in AddressIdentification
  with open('output\\blacklistAddressesOSMIH.pickle', 'wb') as export:
    pickle.dump(otherServicesListDf, export)

# EXECUTE: WalletClustering_AddressIdentification notebook

In [1]:
%run ./WalletClustering_AddressIdentification.ipynb # includes list creation of addresses
# methods & variables of notebook can be referenced

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\rankl\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\rankl\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\Users\rankl\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.
                                     address              association
0         19hx1B4pTDsBAYJJYYrj1t6bcVw1e9omuH                huobi.com
1         16vyeuwbuNz9PKPs4VqqGLsVRJ2G6uJj39                huobi.com
2         12uuCfvuxeyGhWaEU1sHTbkCN7eWMT7qFp                huobi.com
3         1NMbcRqggWu75DG5AqZtin5kecdHoDVs3b                huobi.com
4         1DgcQFeaCQBo41oxtzUuV3caW5TNjXvhLZ                huobi.com
...                                      ...                      ...
13943141  1JiTqsNvfjkGSYVHMKHpWbNwEfN2LUhjKM  unknown_large_wallet_03
13943142  1HoNkBpNPkcznGWe6nUeLqffeLQN1TwhFJ  unknown_large_wallet_03
13943143  1BmvAcQPLcFve9KpNigfF591PyQnZwNDyQ  unknown_large_wallet_03
13943144  1GNw9S6hSzVQ83gUkP36ypjyrH3d6YZRm7  unknown_large_wallet_03
13943145  1McAmDN95SGxaTMmymaN8QzzTGJADM3aZy  unknown_large_wallet_03

[13933603 rows x 2 columns]
['12sDU3FyYJXc2oRzE6XXuuhVHCBJvaoCC8', '1348ThkNoDupq1bws95diMiL8

In [2]:
# only to remember lists
blacklistAddresses = pickle.load(open('output/blacklistAddresses.pickle', 'rb')) # list of exchange wallet addresses

# DEFINE: update associations in neo4j

In [3]:
#checks if there is already an association in place. If yes, skip entry
check_association = """Match (a:Address {address: '%s'}) 
where a.association is not null
return true"""

In [4]:
def updateAssociations(inputDf):
    associations = list(inputDf['association'].unique())
    
    for item in associations:
       
        projection = inputDf[inputDf['association'].str.match(item)]
        addresses = list(projection["address"].values)
        #response = conn.query(check_association % addresses[0],db='neo4j')
        #if not response:
        query = """CALL apoc.periodic.iterate( 'UNWIND $addresses as item return item',
                        'Match (a:Address {address: item}) set a.association = "%s" return a', 
                        {batchSize:1000, parallel:true, iterateList:true, params:{addresses:%s}})""" % (item, str(addresses))
        result = conn.query(query,db='neo4j')
        #else:
        #    print(addresses[0] + ": There is already an association in the DB")

# EXECUTE: update associations in neo4j

In [5]:
updateAssociations(blacklistAddresses)